このファイルについて
- about: 生の検索履歴データから異常なレコードを除去する前処理を行う
- author: 松永

In [1]:
import os
from functools import reduce
import datetime as dt
import pandas as pd
import cudf
import warnings
warnings.simplefilter('ignore')

### ファイル名などの定数宣言

In [2]:
ORIGIN_DATA_DIR = '../../Input_original_data'
PROCESSED_DATA_DIR = '../../Input_processed_data'

IC_CSV = f'{PROCESSED_DATA_DIR}/road_master/ic_merged.csv'

ORIGIN_SEARCH_LOG_DIR = lambda month: f'{ORIGIN_DATA_DIR}/search_log/csv{month}'
ORIGIN_SEARCH_LOG_CSV_LIST = lambda date: [
    f'{ORIGIN_SEARCH_LOG_DIR(date[:6])}/DorapuraSearchLog-{date}.csv',
    f'{ORIGIN_SEARCH_LOG_DIR(date[:6])}/ExternalSvrSearchLog-{date}.csv',
    f'{ORIGIN_SEARCH_LOG_DIR(date[:6])}/SwitchSvrSearchLog-{date}.csv'
]

PROCESSED_SEARCH_LOG_DIR = lambda month: f'{PROCESSED_DATA_DIR}/search_records/csv{month}'
PROCESSED_SEARCH_LOG_CSV = lambda date: f'{PROCESSED_SEARCH_LOG_DIR(date[:6])}/record_{date}.csv'

SEARCH_LOG_COLNAME = [
    'date', 'start_code', 'start_name', 'end_code', 'end_name',
    'via1_code', 'via1_name', 'via2_code', 'via2_name', 'via3_code', 'via3_name',
    'spec_day', 'spec_time', 'spec_type', 'order', 'car_type', 'use_nexco', 'use_urban', 'use_local',
    'language'
]

### 準備

#### 時刻操作系関数

In [3]:
def str2date(date_str, format='%Y%m%d'):
    '''
    文字列形式の日付 => datetime.Dateクラスの日付

    Parameters
    ----------
    date_str: str
    日付
    format: str
    日付文字列の形式

    Returns
    -------
    date: datetime.date
    '''
    date = dt.datetime.strptime(date_str, format).date()
    return date

def str2time(time_str, format='%H:%M'):
    '''
    文字列形式の時刻 => datetime.Timeクラスの時刻
    Parameters
    ----------
    time_str: str
    時刻
    format: str
    時刻文字列の形式

    Returns
    -------
    time: datetime.time
    '''
    time_datetime = dt.datetime.strptime(time_str, format)
    time = time_datetime.time()
    return time

#### ICコード --> IC名

In [5]:
df_ic = pd.read_csv(IC_CSV, dtype={'ic_code': str})
code2name = dict(zip(df_ic['ic_code'], df_ic['ic_name']))
name2code = {name: code for code, name in code2name.items()}

### オリジナルデータ加工 -> CSV

In [6]:
def proc_spec_day(x):
    try:
        # dateに変換できるかどうか確認
        return str2date(x, format='%Y/%m/%d')
    except (ValueError, TypeError) as e:
        # print(e, x)
        return
    
def proc_spec_time(x):
    try:
        # dateに変換できるかどうか確認
        return str2time(x)
    except (ValueError, TypeError) as e:
        print(e, x)
        return


def preprocessing(df):
    # ICコードに”E”が含まれている行を除去
    df = df[df['start_code'].str.contains('E') == False]
    df = df[df['end_code'].str.contains('E') == False]

    # マスタデータに含まれないICコードが誤って記録されている行を除去
    df = df[df['start_code'].apply(lambda c: c in code2name)]
    df = df[df['end_code'].apply(lambda c: c in code2name)]

    # マスタデータに含まれないIC名称が誤って指定されている行を除去
    df = df[df['start_name'].apply(lambda n: n in name2code)]
    df = df[df['end_name'].apply(lambda n: n in name2code)]

    # 指定日に異常な日付が入っている行（e.g. 2021/04/31, 210327）を除去
    df['spec_day'] = df['spec_day'].apply(proc_spec_day)
    df = df[df['spec_day'].notna()]

    # 指定時間が”24:00”となっている行の指定日・指定時間を翌日に繰り上げ
    df['spec_day'] = df['spec_day'].mask(df['spec_time'] == '24:00', df['spec_day'] + dt.timedelta(days=1))
    df.replace({'spec_time': {'24:00': '0:00'}}, inplace=True)

    # 指定時間が\"24:12\"のように異常な行を除去
    df = df[df['spec_time'].str[:2] != '24']

    # 「指定言語」列を除去
    df.drop('language', axis=1, inplace=True)

    df.reset_index(drop=True, inplace=True)

    return df

In [7]:
# test
df = pd.read_csv(ORIGIN_SEARCH_LOG_CSV_LIST('20240801')[0],
          header=0, names=SEARCH_LOG_COLNAME,
          dtype={'start_code': str, 'end_code': str, 'via1_code': str, 'via2_code': str, 'via3_code': str})
df.head()

,date,start_code,start_name,end_code,end_name,via1_code,via1_name,via2_code,via2_name,via3_code,via3_name,spec_day,spec_time,spec_type,order,car_type,use_nexco,use_urban,use_local,language
0,2024/08/01 00:00:00,1050176,滝川,1050191,旭川鷹栖,NaN,NaN,NaN,NaN,NaN,NaN,2024/07/28,09:20,1,2,2,1,1,1,0
1,2024/08/01 00:00:00,1110350,三郷中央,5007026,逗子,NaN,NaN,NaN,NaN,NaN,NaN,2024/07/31,00:00,1,2,2,1,1,1,0
2,2024/08/01 00:00:00,1120031,千葉北,1020301,宇部ＪＣＴ,NaN,NaN,NaN,NaN,NaN,NaN,2024/08/07,00:00,1,2,2,1,1,1,0
3,2024/08/01 00:00:01,5007041,衣笠,6011036,北池袋,NaN,NaN,NaN,NaN,NaN,NaN,2024/07/31,13:00,1,2,2,1,1,1,0
4,2024/08/01 00:00:01,1101151,中津川,1910290,高槻,NaN,NaN,NaN,NaN,NaN,NaN,2024/08/05,10:00,1,2,1,1,1,1,0


## main

In [10]:
start_date='20240507'
end_date= '20240831'

DAYS = [d.strftime('%Y%m%d') for d in pd.date_range(start_date, end_date, freq='1D')]

dtypes = {
    'start_code': str, 
    'end_code': str,
    'via1_code': str, 
    'via2_code': str,
    'via3_code': str,
}

for d in DAYS:
    df_day = None
    for csv in ORIGIN_SEARCH_LOG_CSV_LIST(d):
        if not os.path.exists(csv):
            continue

        df = pd.read_csv(csv, header=0, names=SEARCH_LOG_COLNAME, dtype=dtypes)
  
        df = preprocessing(df)
        print(os.path.basename(csv), len(df))

        df_day = pd.concat([df_day, df], ignore_index=True, copy=False)

    print('-'*30, f'[{d}] {len(df_day)} records', '-'*30)

    df_day.to_csv(PROCESSED_SEARCH_LOG_CSV(d), index=False)

print('='*40, 'finished', '='*40)

DorapuraSearchLog-20240507.csv 549405
ExternalSvrSearchLog-20240507.csv 1413
SwitchSvrSearchLog-20240507.csv 3210
------------------------------ [20240507] 554028 records ------------------------------
DorapuraSearchLog-20240508.csv 465817
ExternalSvrSearchLog-20240508.csv 1330
SwitchSvrSearchLog-20240508.csv 2332
------------------------------ [20240508] 469479 records ------------------------------
DorapuraSearchLog-20240509.csv 483882
ExternalSvrSearchLog-20240509.csv 1549
SwitchSvrSearchLog-20240509.csv 2251
------------------------------ [20240509] 487682 records ------------------------------
DorapuraSearchLog-20240510.csv 492643
ExternalSvrSearchLog-20240510.csv 1248
SwitchSvrSearchLog-20240510.csv 2033
------------------------------ [20240510] 495924 records ------------------------------
DorapuraSearchLog-20240511.csv 412876
ExternalSvrSearchLog-20240511.csv 1153
SwitchSvrSearchLog-20240511.csv 1556
------------------------------ [20240511] 415585 records ---------------------